### Importing essential libraries 

In [2]:
import pandas as pd 
import numpy as np 
import requests 
from bs4 import BeautifulSoup


link = 'https://github.com/AshishJangra27'

soup = BeautifulSoup(requests.get(link).content,'html.parser')

### Defining a social account functions 

In [3]:
def social_account(urls):
    accounts = []
    for url in urls:
        if 'kaggle' in url:
            accounts.append('Kaggle')
        elif 'instagram' in url:
            accounts.append('Instagram')
        elif 'facebook' in url:
            accounts.append('Facebook')
        elif 'linkedin' in url:
            accounts.append('Linkedin')
        elif 'github' in url:
            accounts.append('Github')
        else:
            accounts.append('Personal website')
    return ','.join(accounts)

### Scraping the data of the github repositories 

In [4]:
user_name  = soup.find('span',class_ = 'p-name vcard-fullname d-block overflow-hidden').text.strip()
user_id    = soup.find('span',class_ = 'p-nickname vcard-username d-block').text.strip()
bio        = soup.find('div',class_ = 'js-profile-editable-area d-flex flex-column d-md-block').find('div').text
lives_in   = soup.find('li',class_ = 'vcard-detail pt-1 hide-sm hide-md').text.strip()
data = []
for i in soup.find_all('a',class_ = 'Link--secondary no-underline no-wrap'):
    if 'follower' in (i.text):
        follower = int(i.find('span').text.strip())
    elif 'following' in i.text:
        following = int(i.find('span').text.strip())
    try:        
        content  = [int(content.find('span').text.strip()) for content in soup.find('nav',class_ ='UnderlineNav-body width-full p-responsive js-sidenav-container-pjax').find_all('a')[1:]]
        repos    = content[0]
        project  = content[1]
        packages = content[2]
        stars    = content[3]
    except:
        repos    = np.nan
        project  = np.nan
        packages = np.nan
        stars    = np.nan
    try:
        url = [links.find('a')['href'] for links in soup.find_all('li',class_='vcard-detail pt-1')]
        account = social_account(url)
    except:
        url = np.nan
        account = 'no social account has been linked'


data.append([user_id , user_name , bio , follower , following ,lives_in,url,account, repos , project , packages , stars])

df = pd.DataFrame(data,columns = ['user_id' ,'user_name' ,'bio' ,'follower' ,'following', 'lives_in', 'url','account', 'repos' , 'project' , 'packages' , 'stars'])
df.head()




,user_id,user_name,bio,follower,following,lives_in,url,account,repos,project,packages,stars
0,AshishJangra27,Ashish Jangra,I am an AI Trainer. \nI train students on tech...,300,1,Noida,[],,72,0,0,52


### Scraping the repos

In [5]:
link = 'https://github.com/AshishJangra27?tab=repositories'
res = requests.get(link)
soup = BeautifulSoup(res.content,'html.parser')

repos_links = ','.join(['https://www.github.com' + name.find('a').get('href') for name in soup.find_all('div',class_ ='d-inline-block mb-1')])
repos_name  = ','.join([name.find('a').text.strip() for name in soup.find_all('div',class_ ='d-inline-block mb-1')])


### Concating the data 


In [10]:
df_ = pd.DataFrame()
df_['repos_name'] = [repos_name]
df_['repos_links'] = [repos_links]
df = pd.concat([df,df_],axis = 1)
df

,user_id,user_name,bio,follower,following,lives_in,url,account,repos,project,packages,stars,repos_name,repos_links
0,AshishJangra27,Ashish Jangra,I am an AI Trainer. \nI train students on tech...,300,1,Noida,[],,72,0,0,52,"Skill-India-AI-ML-Scholarship,NLP-Course-GFG,D...",https://www.github.com/AshishJangra27/Skill-In...


### Scraping the stars

In [42]:
link = 'https://github.com/AshishJangra27?tab=stars'
star_data = []
for pages in range(2):
    res = requests.get(link)
    soup = BeautifulSoup(res.content,'html.parser')
    stars_repo_link = ['https://www.github.com' + stars.find('a').get('href') for stars in soup.find_all('div',class_ = 'd-inline-block mb-1')]
    stars_repo_name = [stars.find('a').text.strip().split('/')[1].strip() for stars in soup.find_all('div',class_ = 'd-inline-block mb-1')]

    star_data += stars_repo_name,stars_repo_link
    link = soup.find('div',class_ = 'BtnGroup').find('a').get('href')

30

In [41]:
link

'https://github.com/AshishJangra27?after=Y3Vyc29yOnYyOpK0MjAyMi0wNi0wOVQwNzoxMTozM1rOFEo6ow%3D%3D&tab=stars'